<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 5: cadCAD Configuration

---

# Overview

We're ready to configure our first model and simulation! We'll introduce a "Toy Model" of an ecosystem, before applying a modelling and simulation framework to learn the core cadCAD features and modelling process.

## Learning Outcome

* Learn about the core cadCAD features and configuration process
* Introduce a modelling and simulation framework that you can practically apply to many system modelling scenarios

## Table of Contents

* [Toy Model Introduction](#Toy-Model-Introduction:-An-Ecosystem-Model)
  * [Some Context: Lotka & Volterra](#Some-Context:-Lotka-&-Volterra)
  * [Visual System Mapping: Entity Relationship Diagram](#Visual-System-Mapping:-Entity-Relationship-Diagram)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Mathematical Specification](#Mathematical-Specification)
  * [Visual System Mapping: Stock & Flow Diagram](#Visual-System-Mapping:-Stock-&-Flow-Diagram)
  * [Differential Specification](#Differential-Specification)
* [cadCAD Standard Notebook Layout](#cadCAD-Standard-Notebook-Layout)
    0. [Dependencies](#0.-Dependencies)
    1. [State Variables](#1.-State-Variables)
    2. [System Parameters](#2.-System-Parameters)
    3. [Policy Functions](#3.-Policy-Functions)
    4. [State Update Functions](#4.-State-Update-Functions)
    5. [Partial State Update Blocks](#5.-Partial-State-Update-Blocks)
    6. [Configuration](#6.-Configuration)
    7. [Execution](#7.-Execution)
    8. [Simulation Output Preparation](#8.-Simulation-Output-Preparation)
    9. [Simulation Analysis](#9.-Simulation-Analysis)

# Toy Model Introduction: An Ecosystem Model

> Ecosystem: a biological community of interacting organisms and their physical environment.

<center>
<img src="./images/ecosystem.png"
     alt="Ecosystem"
     style="width: 200px;" />
</center>

## Some Context: Lotka & Volterra

<center>
<img src="./images/lotka-volterra.png"
     alt="Lotka-Volterra"
     style="width: 60%;" />
</center>

## Visual System Mapping: Entity Relationship Diagram
<!-- https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model -->

<center>
<img src="./images/basic-flow-chart.png"
     alt="Flow chart"
     style="width: 400px;" />
</center>

## Requirements Analysis

TODO: mention business requirements, technical requirements, etc.

Illustrative real-world model applications:
* Forecast animal food consumption, to determine the sustainability of a farming operation, and to plan for worst-case scenarios.
* Given a food supply of a number of standard crops, of varying cost, how do we optimize economic performance of the farming operation?
* Given the ecological impact of a certain crop on the fertility of the soil, how do we balance the economic performance and ecological sustainability of the farming operation?

### Investigation (TODO: or What-if questions?)

1. How...?
2. When...?
3. What if...?

### Assumptions (TODO: update script accordingly)

1. The population will increase over time.
2. The food supply will decrease over time.
3. There is some relationship between the population and food supply.

## Mathematical Specification

> ...differential equations play a prominent role in many disciplines including engineering, physics, economics, and biology.

### Differential Equations
* A population consumes a food source, and reproduces at a rate proportional to the food source.
* The food source is consumed at a rate proportional to the population.

\begin{align}
population_t &= population_{t-1} + {\Delta population} \quad &\textrm{(sheep/month)} \tag{1} \\
food_t &= food_{t-1} + {\Delta food} \quad &\textrm{(tons of grass/month)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
{\Delta population} &= \alpha * food_{t-1} \quad &\textrm{(sheep/month)} \\
{\Delta food} &= -\beta * population_{t-1} \quad &\textrm{(tons of grass/month)}
\end{align}

## Visual System Mapping: Stock & Flow Diagram

<center>
<img src="./images/s5-stock-and-flow-ecosystem.png"
     alt="Stock and flow"
     style="width: 600px;" />
</center>

## Differential Specification

<center>
<img src="./images/s5-differential-spec-ecosystem.png"
     alt="Differential spec"
     style="width: 400px;" />
</center>

# cadCAD Standard Notebook Layout

<center>
<img src="./images/cadcad-flow.png"
     alt="cadCAD flow"
     style="width: 25%;" />
</center>

# 0. Dependencies

In [1]:
# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd
# import plotly

In [2]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# 1. State Variables

> A state variable is one of the set of variables that are used to describe the mathematical "state" of a dynamical system. ([Wikipedia](https://en.wikipedia.org/wiki/State_variable))

In [3]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}
initial_state

## **Time** as a system state

<center>
<img src="./images/discrete-time.svg"
     alt="Discrete time"
     style="width: 200px;" />
</center>

* 1 **timestep** == 1 month

# 2. System Parameters

> System parameterization is the process of choosing variables that impact the behaviour of the model. These parameters allow us to perform simulation techniques like parameter sweeps, Monte Carlo simulations, A/B tests, and see how the system behaves under a different model parameter set.

In [4]:
system_params = {
    'reproduction_rate': [0.01], # sheep per month
    'consumption_rate': [0.01], # tons of grass per month
}
system_params

# 3. Policy Functions

> A Policy Function computes one or more signals to be passed to State Update Functions. They describe the logic  and behaviour of a system component  or mechanism.

We'll cover this in the next section!

# 4. State Update Functions

> We create State Update Functions to design the way our model state changes over time. These will usually represent the system differential specification.

```python
def state_update_function(params, substep, state_history, previous_state, policy_input):
    variable_value = 0
    return 'variable_name', variable_value
```

* `params` is a Python dictionary containing the **system parameters** <!-- for consistency with the previous definition -->
* `substep` is an integer value representing a step within a single `timestep`
* `state_history` is a Python list of all previous states
* `previous_state` is a Python dictionary that defines what the state of the system was at the **previous timestep** or **substep**
* `policy_input` is a Python dictionary of signals or actions from **policy functions**

In [5]:
def update_population(current_population, alpha, food_supply):
    """
    Update the population state according to the differential equation (1):
    current_population + alpha * food_supply
    """
    return math.ceil(current_population + alpha * food_supply)

math.ceil(5.5)

In [6]:
# Relevant state variables
current_population = initial_state['population']
food_supply = initial_state['food']

# Relevant parameters
reproduction_rate = system_params['reproduction_rate'][0] # "alpha" in our differential equation

update_population(current_population, reproduction_rate, food_supply)

In [7]:
def s_population(params, substep, state_history, previous_state, policy_input):
    """
    Update the population state according to the differential equation (1):
    current_population + alpha * food_supply
    """
    population = previous_state['population'] + params['reproduction_rate'] * previous_state['food']
    return 'population', max(math.ceil(population), 0)

In [35]:
population = 60
print("A tuple!")
'population', max(math.ceil(population), 0)

In [34]:
next_state = {
    # current_population + alpha * food_supply
    'population': math.ceil(50 + 0.01 * 1000),
    'food': 1000
}
next_state

In [10]:
def s_food(params, substep, state_history, previous_state, policy_input):
    """
    Update the food supply state according to the differential equation (2):
    current_food - beta * population
    """
    food = previous_state['food'] - params['consumption_rate'] * previous_state['population']
    return 'food', max(food, 0)

In [11]:
max(-10, 0)

# 5. Partial State Update Blocks
## Tying it all together

> A series of Partial State Update Blocks is a structure for composing State Update Functions and Policy Functions in series or parallel, as a representation of the system model. 

<center>
<img src="./images/basic-psub.png"
     alt="Policy functions"
     style="width: 70%;" />
</center>

**Updates run in series**

In [12]:
partial_state_update_blocks = [
    # Run first
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            'population': s_population
        }
    },
    # Run second
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            'food': s_food
        }
    }
]

**Updates run in parallel**

In [13]:
partial_state_update_blocks = [
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            # Updated in parallel
            'population': s_population,
            'food': s_food
        }
    }
]

# 6. Configuration

> The configuration stage is about tying all the previous model components together and choosing how the simulation should run.

Configuration parameters:
* `'N': 1` - the number of times we'll run the simulation (you'll see them called "Monte Carlo runs" later in the course, when we look at tools to analyze system models)
* `'T': range(400)` - the number of timesteps the simulation will run for
* `'M': system_params` - the parameters of the system

In [14]:
sim_config = config_sim({
    "N": 1,
    "T": range(400),
    "M": system_params
})

In [15]:
range(400)

In [16]:
list(range(400))

In [17]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [18]:
exp = Experiment()
exp.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

# 7. Execution

> The Execution Engine takes a model and configuration, and computes the simulation output.

## Configuring the cadCAD simulation execution

In [19]:
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

In [20]:
simulation = Executor(exec_context=local_mode_ctx, configs=configs)

## Time to simulate our ecosystem model!

In [21]:
raw_result, tensor_field, sessions = simulation.execute()

# 8. Simulation Output Preparation
> The simulation results are returned as a Pandas dataframe. At this stage of the process you'll manipulate and analyze your results to answer questions about your model.

In [22]:
simulation_result = pd.DataFrame(raw_result)

In [23]:
raw_result[:5]

In [24]:
simulation_result.head()

# 9. Simulation Analysis

In [25]:
pd.options.plotting.backend = "plotly"

In [26]:
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

In [27]:
pd.set_option('display.max_rows', len(simulation_result))
display(simulation_result)
pd.reset_option('display.max_rows')

In [28]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
simulation_result.query('food == 0').head()

# Well done! Next section: System Analysis